In [8]:
# Make sure to pip install pyarrow and fastparquet
import pandas as pd
import pickle

In [2]:
data = pd.read_csv('troop_movements_1m.csv')

print(data.head())

             timestamp  unit_id     unit_type  location_x  location_y  \
0  2023-06-03 03:19:15   919214  tie_silencer         2.0         5.0   
1  2023-02-24 13:50:40  9467154  stormtrooper         9.0         0.0   
2  2023-03-29 19:54:55  6585778  tie_silencer         0.0         6.0   
3  2023-04-30 00:58:11  3878023  tie_silencer         4.0         2.0   
4  2023-04-10 22:00:26  5537117         at-st         6.0         8.0   

   destination_x  destination_y    homeworld  
0              9              3  Aleen Minor  
1              9              1    Malastare  
2              5              9      Serenno  
3              9              9         Tund  
4              5              8        Skako  


In [3]:
data['unit_type'].str.replace('invalid_unit', 'unknown')

0         tie_silencer
1         stormtrooper
2         tie_silencer
3         tie_silencer
4                at-st
              ...     
999995     tie_fighter
999996           at-at
999997     tie_fighter
999998     tie_fighter
999999           at-st
Name: unit_type, Length: 1000000, dtype: object

In [4]:
data['location_x'] = data['location_x'].ffill()
data['location_y'] = data['location_y'].ffill()

In [5]:
data.to_parquet("troop_movements_1m.parquet")
data_loaded = pd.read_parquet("troop_movements_1m.parquet")

In [18]:
with open("trained_model.pkl", "rb") as file:
    model = pickle.load(file)

X = pd.get_dummies(data_loaded[['homeworld', 'unit_type']])

X = X.reindex(columns=model.feature_names_in_, fill_value=0)

data_loaded['predictions'] = model.predict(X)

In [19]:
data_loaded.head()

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld,predictions
0,2023-06-03 03:19:15,919214,tie_silencer,2.0,5.0,9,3,Aleen Minor,False
1,2023-02-24 13:50:40,9467154,stormtrooper,9.0,0.0,9,1,Malastare,False
2,2023-03-29 19:54:55,6585778,tie_silencer,0.0,6.0,5,9,Serenno,False
3,2023-04-30 00:58:11,3878023,tie_silencer,4.0,2.0,9,9,Tund,False
4,2023-04-10 22:00:26,5537117,at-st,6.0,8.0,5,8,Skako,False
